In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

# Setting the pyecharts config
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

# setting pyecharts' charts
from pyecharts.charts import Funnel, Bar, Line, Grid
from pyecharts import options as opts
from pyecharts.globals import ThemeType

# Jupyter Magic Func
%matplotlib inline

In [19]:
#importing data
df = pd.read_csv('Data/UserBehavior.csv', nrows=1000000, header=None, parse_dates=True)

# rename columns
df.columns = ['User_Id', 'Item_Id', 'Category_Id', 'Behavior_Type', 'Time']

# trans the dtype
df['Time'] = pd.to_datetime(df['Time'], unit='s')
df['Time'] = df.Time.astype('str')

#split the columns
df['Time_Split'] = df.Time.str.split(' ')
df['Date'] = df.Time_Split.str.get(0)
df['Hour'] = df.Time_Split.str.get(1)

# Drop the Column
df = df.drop(['Time_Split'], axis=1)

# Redefine the order of columns 
df = df[['Date', 'Hour', 'Time', 'User_Id', 'Item_Id', 'Category_Id', 'Behavior_Type']]

# Tranform the dtype back to datetime, in order to split the "Date" Column to two columns.
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'])

# Clean the wrong data range. Ex. outer the range from 2017,11,23 to 2017,12,24.
mask_1 = df['Date'] >= pd.datetime(2017,11,24)
mask_2 = df['Date'] <= pd.datetime(2017,12,24)
df = df[mask_1 & mask_2]

---
# Alibaba User Behavior 分析報告：

## A. 問題挖掘
* 利用電商常用模型 AARRR 與 RFM ，來分析各階段的留客率並挖掘洞見，找出能夠改善的環節並優化。
* 利用時間規律，找出用戶的活躍傾向，並藉此規劃行銷企劃。
* 分析用戶對不同商品的偏好程度，進一步設計各個商品的行銷計畫。
* 找到核心的付費用戶群，並據此進行用戶畫像的繪製。

## B. 資料概述
* 時間：2017 年 11 月 23 日至 2017 年 12 月 24 日
* 資料類別：
1. 用戶 ID
2. 物品 ID
3. 品類 ID
4. 行為類別
5. 時間戳記

## C. 分析細節

### 1. 分析參考模型
1. AARRR Model
2. RFM Model

### 2. AARRR｜漏斗模型分析
平均每個用戶在這 31 天的數據，共有 102.33 次的月瀏覽量，平均每日的瀏覽量共有 3.2 次。

#### i. Activation & Retention｜活躍度與留存分析
術語說明｜跳離率：僅一次點擊的用戶總數 / 用戶總量。

**結論**：淘寶擁有足夠的吸引力，讓用戶停留在 APP 中。
1. 資料統計為 32 天裡，983 個用戶中只有 55 個人瀏覽過一次便跳離，佔總訪問人數僅有 0.281%。
2. 平均每位用戶每天瀏覽淘寶 APP 10.17 次，說明用戶對淘寶服務的黏著度極高，基本為每日必使用的 APP。

#### ii. 用戶行為漏斗
**結論**：瀏覽到加入 購物車 / 我的最愛 環節明顯有提高的空間，是提升指標的重要環節。

因為放入購物車和加入我的最愛，都屬於用戶展示購買意願的階段，且因用戶習慣不同並無特別順序，因此將兩者皆算做同階段，並繪製漏斗圖展示轉換率。
1. 可以看到從瀏覽到購買僅有 9.13% 的轉化率，當然也會有部分用戶是跳過加入最愛直接購買。
2. 從我的最愛與購物車中轉化至購買的比例則有 25.65%，其中加入購物車（Cart）的比例為 66.5% 明顯高於我的最愛（Fav），顯示多數用戶多會直接加入購物車。
</b></b>

#### iii. 用戶行為分析解構：（後面再來分析）
1. 瀏覽 -> 加入購物車 -> 購買  
2. 瀏覽 -> 購買
3. 瀏覽 -> 加我的最愛 -> 加入購物車 -> 購買
4. 瀏覽 -> 加入購物車 -> 購買

### 3. 不同時間尺度下用戶傾向
**分析主軸**：此部分嘗試以不同的時間尺度下，分析用戶的行為模式與活躍規律。

#### i. 分析一個月中用戶行為規律
此處以月為週期觀察，可以發現整體用戶呈現週期性的漲跌。且瀏覽數量基本上隨著用戶的購買行為起伏。

其中，加入我的最愛與購物車為異步行為，因此在購買行為發生前不久才會出現。同時
另外加入購物車後，
且放入購物車為購買的前置動作，因此在週末結帳前會有提升的趨勢，直到假日購買量才會上升。

In [174]:
attr = ['Date', 'buy', 'cart', 'fav', 'pv']
df_one_month = pd.pivot_table(df, index="Date", columns="Behavior_Type", aggfunc=['count']).iloc[:30,:4].reset_index()
df_one_month.columns = attr

# Extract the column to the list
user = []
for stage in attr:
    user.append(df_one_month[stage].astype('str').to_list())

    
def monthly_user_behavior():
    '''
    Create the monthly chart to analyze the user behavior in one month.
    '''
    user_bar = (
        Bar()
        .add_xaxis(user[0])
        .add_yaxis('購買人數', user[1], stack='stack1', category_gap=40)
        .add_yaxis('放入購物車', user[2], stack='stack1', category_gap=40)
        .add_yaxis('我的最愛', user[3], stack='stack1', category_gap=40)
        .extend_axis(
                yaxis=opts.AxisOpts(
                    axislabel_opts=opts.LabelOpts(formatter="{value} 人"), interval=20000
                )
            )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="用戶變化週期曲線"),
            yaxis_opts=opts.AxisOpts(
                axislabel_opts=opts.LabelOpts(formatter="{value} 人"), interval=20000
            ),
#             datazoom_opts=opts.DataZoomOpts(orient="vertical"),
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    )

    user_line = (
        Line()
        .add_xaxis(user[0])
        .add_yaxis('購買人數', user[1], stack='stack2', yaxis_index=1)
        .add_yaxis('放入購物車', user[2], stack='stack2', yaxis_index=1)
        .add_yaxis('我的最愛', user[3], stack='stack2', yaxis_index=1)
        .add_yaxis('購買', user[4], stack='stack2', yaxis_index=1)
    )
    # overlap two charts
    user_bar.overlap(user_line)
    
    #return the chart
    return user_bar

# user_bar.render_notebook()
monthly_user_behavior().render_notebook()

#### ii. 分析一週中用戶行為規律
時間：2017-11-27 (Monday) ~ 2017-12-03 (Sunday)
我們取雙十二前一週進行參考，可以發現禮拜四為各指標最低的一天，隨後持續上漲至禮拜六為高峰。推測是禮拜五開始上班族下班後開始頻繁使用淘寶，而週六有充足的時間與精神進行購買行為。
對於瀏覽量，可以看見週一瀏覽量明顯較低，且禮拜日的我的最愛明顯較高。因此若要維持用戶活躍，能於禮拜一進行少量推播，並根據用戶所加入的我的最愛，進行個性化推播，應能吸引用戶維持黏著度。

##### **建議**：
* 平時可以把行銷活動重點資源，投注在禮拜五至假日。
* 於週一進行少量推播，並根據用戶的我的最愛進行個性化推播，維持用戶活躍習慣。



# D.不同商品種類的用戶行為
統計所有被購買的商品，可以發現單項商品最高僅被購買 4 次，沒有特別集中的銷售商品，說明商品銷售多依靠長尾效應銷售，而非特殊熱門商品帶動營收。

下列比較前 10 名的購買與瀏覽商品，可以發現兩者幾乎沒有正向關係。銷售量第一名的產品甚至沒有排進瀏覽量的前十名，說明瀏覽量沒有很好的轉化為實際購買量。

In [211]:
items_sale = df.query('Behavior_Type == "buy"').groupby('Item_Id')['Item_Id'].count().sort_values(ascending=False)[:10].tolist()
items_sale_id = df.query('Behavior_Type == "buy"').groupby('Item_Id')['Item_Id'].count().sort_values(ascending=False)[:10].index.to_list()
items_view = df.query('Behavior_Type == "pv"').groupby('Item_Id')['Item_Id'].count().sort_values(ascending=False)[:10].to_list()
items_view_id = df.query('Behavior_Type == "pv"').groupby('Item_Id')['Item_Id'].count().sort_values(ascending=False)[:10].index.to_list()
items_cart_id = df.query('Behavior_Type == "cart"').groupby('Item_Id')['Item_Id'].count().sort_values(ascending=False)[:20].index.to_list()

In [216]:
def item_sales() -> Grid:
    bar = (
        Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
        .add_xaxis(items_sale_id)
#         .add_yaxis("銷售量", items_view)
        .add_yaxis("銷售量", items_sale)
#         .extend_axis(
#                 yaxis=opts.AxisOpts(
#                     axislabel_opts=opts.LabelOpts(formatter="{value} 人"), interval=20000
#                 )
#             )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="商品購買排行"),
#             yaxis_opts=opts.AxisOpts(
#                 axislabel_opts=opts.LabelOpts(formatter="{value} 人"), interval=2000
#             ),
# #             datazoom_opts=opts.DataZoomOpts(orient="vertical"),
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
    )
        
    bar_view = (
        Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
        .add_xaxis(items_view_id)
        .add_yaxis("瀏覽人數", items_view)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="商品瀏覽排行", pos_top="48%"),
            legend_opts=opts.LegendOpts(pos_top="48%"),
        )
    )
    
    grid = (
        Grid()
        .add(bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
        .add(bar_view, grid_opts=opts.GridOpts(pos_top="60%"))
    )
    
    return grid

item_sales().render_notebook()

## 3. RFM 模型分析
透過 RFM 模型找出對營收貢獻最大的用戶，能夠透過該用戶數據，進行用戶畫像的描繪。在本數據中，因商品與營收等資料都已匿名化，因此主要分析 R（Recency）與 F（Frequency）進行分析。

### i. 最近購買時間
Rencency 會根據
會將近一個月的顧客分成五等分，每一等分為資料庫的 20%。
該分析便是找出至今再次購買與上次購買產品的時間差的 20% 用戶。換句話說，距上次消費時間越近的用戶，會被分到更高的等級。

In [246]:
df_rfm = df.groupby('User_Id')['Date'].max().reset_index()
df_R = (pd.to_datetime("2017-12-03") - pd.to_datetime(df_rfm.Date)).dt.days
df_R.unique()

array([0, 1, 2, 4, 3, 6])